<a href="https://colab.research.google.com/github/hargurjeet/LJMU_Thesis/blob/main/TabToText_Exp22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 734.6 kB/s eta 0:00:00


In [7]:
import pandas as pd
import openai
openai.api_key = ''

In [4]:
travel_data = pd.read_csv('/content/Customertravel.csv')
travel_data.head()

,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target
0,34,No,Middle Income,6,No,Yes,0
1,34,Yes,Low Income,5,Yes,No,1
2,37,No,Middle Income,3,Yes,No,0
3,30,No,Middle Income,2,No,No,0
4,30,No,Low Income,1,No,No,0


# Experiment 1

# Experiment 2

In [33]:
dummy_df = travel_data.drop(columns='Target', axis=1).head()
dummy_df = travel_data.head(100).copy()
dummy_df

,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target
0,34,No,Middle Income,6,No,Yes,0
1,34,Yes,Low Income,5,Yes,No,1
2,37,No,Middle Income,3,Yes,No,0
3,30,No,Middle Income,2,No,No,0
4,30,No,Low Income,1,No,No,0
...,...,...,...,...,...,...,...
95,28,No,Middle Income,2,No,Yes,1
96,30,No,Middle Income,1,Yes,Yes,0
97,31,Yes,Low Income,4,Yes,No,1
98,31,No,Middle Income,3,No,No,0


In [34]:
def row_to_textual_data(row):
    row_str = ', '.join([f"{col}: {val}" for col, val in row.items()])
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"""A Tour & Travels Company Wants To Predict Whether A Customer Will Churn Or Not Based On Indicators Given Below.
                                            Convert the following row to a detailed textual description: {row_str}.
                                            Consider the following constraints:
                                              1.Dont repeat similar discription for every line. No need to follow an order while generation of discription.\
                                              which mean the age can go in last or annual income can be in first...etc.
                                          """}
        ]
    )
    return response['choices'][0]['message']['content']

# Apply the function to each row of the DataFrame and save the results in a new DataFrame
dummy_df['TextualData'] = dummy_df.drop(columns='Target', axis=1).apply(row_to_textual_data, axis=1)

# Convert the Series to a DataFrame
# textual_df = pd.DataFrame(textual_data, columns=['TextualData'])
dummy_df

,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target,TextualData
0,34,No,Middle Income,6,No,Yes,0,"The customer who is 34 years old, with a middl..."
1,34,Yes,Low Income,5,Yes,No,1,The customer is 34 years old and has a low inc...
2,37,No,Middle Income,3,Yes,No,0,The customer is 37 years old and falls into th...
3,30,No,Middle Income,2,No,No,0,This customer is 30 years old with middle inco...
4,30,No,Low Income,1,No,No,0,The customer is 30 years old and does not have...
...,...,...,...,...,...,...,...,...
95,28,No,Middle Income,2,No,Yes,1,The customer is 28 years old and has opted for...
96,30,No,Middle Income,1,Yes,Yes,0,"The customer is 30 years old, does not have fr..."
97,31,Yes,Low Income,4,Yes,No,1,"Based on the provided indicators, let's genera..."
98,31,No,Middle Income,3,No,No,0,This customer is 31 years old and does not hav...


In [35]:
import pandas as pd
import openai
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def get_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response['data'][0]['embedding']

dummy_df['Embedding'] = dummy_df['TextualData'].apply(get_embedding)

In [24]:
len(dummy_df['Embedding'][0])

1536

In [18]:
dummy_df['TextualData'].to_csv('textual_data.csv', index=False)

In [37]:
import pandas as pd
import openai
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assuming dummy_df is already populated with TextualData and Embedding columns

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    dummy_df['Embedding'].tolist(),
    dummy_df['Target'],  # Assuming 'Target' is your target variable
    test_size=0.2,
    random_state=42
)

# Standardize the embeddings
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Dimensionality Reduction with PCA
pca = PCA(n_components=min(len(X_train), X_train.shape[1]))
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_pca, y_train)

# Make predictions and evaluate the model
y_pred = clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Further steps could involve hyperparameter tuning, cross-validation, etc.


Accuracy: 0.85


In [32]:
X_train.shape

(4, 1536)